# How do we prepare Yelp data to answer business questions?

In [1]:
import datetime
import json
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from shapely.geometry import Point, shape

## Goal

In this case, we will introduce you to another uncleaned dataset for practice. We will focus on the nuances of cleaning the dataset parameter by parameter. Being able to dive into a single parameter and investigate its attributes will be crucial for modeling in later stages of the data science proceess.

## Introduction

**Business Context.** [Yelp](https://www.yelp.com) is a very popular website, where anyone can write a review about restaurants, hotels, spas or any business. They have decided to start analysing the data and will use the results of the analysis to make decisions about new features to the service. They have now approached you, an independant data consultant with a requirement of cleaning the data they have and for you to help them get more context on the data.

**Business Problem.** Your task is to go through the data, and make transformations or additions to the data based on their needs, which are listed below.

**Analytical Context.** The client has shared three files with you containing details about the businesses and end-users on the service along with the reviews posted by these users. With this data, they would like you to do the following:

1. For each business, find out the county and state that it is a part of
2. For each business, calculate the total number of reviews received and the average star rating across those reviews
3. For each business, calculate the total number of check-ins by hour of day, by day of week, and across all time
    
We will run these operations locally with a subset of the data. (As in the previous case, in enterprise-grade production one would use a cloud service provider to apply our cleaning script to the entire dataset, but the data file here is quite large so we will only work with a subset of it on our local machines. You are welcome to read more online about these providers if you wish.)

## Reading in the data files

Let's read in all three data files provided and familiarize ourselves with the data. Note that we truncate our read to the first 1000 rows due to the sizes of the files. This means we may have some strange results, such as restaurants having few, if any reviews. This is a great opportunity to practice your data skills! Try reading in larger amounts of data, or only reading reviews for the 1000 businesses you read in.

In [2]:
businesses = pd.read_csv('https://storage.googleapis.com/training-cases-large-case-asset-files/case.data_cleaning_practice/businesses.csv', nrows=1000)
reviews = pd.read_csv('https://storage.googleapis.com/training-cases-large-case-asset-files/case.data_cleaning_practice/reviews.csv', nrows=1000)
checkins = pd.read_csv('https://storage.googleapis.com/training-cases-large-case-asset-files/case.data_cleaning_practice/checkins.csv', nrows=1000)
checkins = checkins.values.tolist()

The columns present in each of the files are:

1. **reviews**
    * **review_id** - Unique identifier for the review
    * **business_id** - Unique identifier of the business that is being reviewed
    * **user_id** - Unique identifier of the user who posted the review
    * **date** - The date and time of the review
    * **star** - Star rating for the review
    * **text** - Review text
    * **cool** - Number of cool votes received for the review
    * **funny** - Number of funny votes received for the review
    * **useful** - Number of funny votes received for the review
    
    
2. **businesses**
    * **business_id** - Unique identifier of the business
    * **name** - Name of the business
    * **categories** - Categories associated with the business
    * **latitude** - Location of the business (latitude)
    * **longitude** - Location of the business (longitude)
    * **review_count** - Number of reviews received for the business
    * **stars** - Average star rating, rounded to half stars
    
    
3. **checkins**
    * **business_id** - Unique identifier of the business
    * **date** - List of datetimes when users checked in to the business

## Handling null values in `reviews`

As always, handling null values is a staple of cleaning datasets. Let's uncover the columns in the table `reviews` that contain null values:

In [3]:
reviews.isna().any()

business_id    False
cool            True
date            True
funny           True
review_id      False
stars           True
text           False
useful          True
user_id        False
dtype: bool

We can see that null values are present in the following columns: `date`, `stars`, `cool`, `funny`, and `useful`. Each of these columns need to be handled in a different way.

In contrast to the previous case, here we will look at some advanced methods for handling null values that don't use normal `pandas` operations. Let's get started.

## `date`

Since the client has indicated they will likely be using the cleaned data for numerous analyses later on, removing rows with missing values or even writing in meaningless filler values will not work. So we need to figure out a sensible interpolation method.


In [4]:
businesses.head(10)

,business_id,categories,city,latitude,longitude,name
0,1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,33.522143,-112.018481,Arizona Biltmore Golf Club
1,QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,43.605499,-79.652289,Emerald Chinese Restaurant
2,gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,35.092564,-80.859132,Musashi Japanese Restaurant
3,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,33.455613,-112.395596,Farmers Insurance - Paul Lorenz
4,HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,35.190012,-80.887223,Queen City Plumbing
5,68dUKd8_8liJ7in4aWOSEA,"Shipping Centers, Couriers & Delivery Services...",Mississauga,43.599475,-79.711584,The UPS Store
6,5JucpCfHZltJh5r1JabjDg,"Beauty & Spas, Hair Salons",Calgary,50.943646,-114.001828,Edgeworxx Studio
7,gbQN7vr_caG_A1ugSmGhWg,"Hair Salons, Hair Stylists, Barbers, Men's Hai...",Las Vegas,36.099872,-115.074574,Supercuts
8,Y6iyemLX_oylRpnr38vgMA,"Nail Salons, Beauty & Spas, Day Spas",Glendale,33.654815,-112.188568,Vita Bella Fine Day Spa
9,4GBVPIYRvzGh4K4TkRQ_rw,"Beauty & Spas, Nail Salons, Day Spas, Massage",Fairview Park,41.440825,-81.854097,Options Salon & Spa


In [5]:
reviews.head(10)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,na4Th5DrNauOv-c43QQFvA,5.0,2004-10-19 02:46:40,3.0,xW294l3Lwh0cxlHU1jwRDA,5.0,The gold standard for casinos everywhere. Just...,5.0,nkN_do3fJ9xekchVC-v68A
1,u8C8pRvaHXg3PgDrsUHJHQ,0.0,2004-10-19 19:24:13,1.0,0QHCY_55TFHHvyumEMpDew,4.0,Good stuff. Pricey by normal pizza standards.,0.0,nkN_do3fJ9xekchVC-v68A
2,EZOoB2D8uQHV_gJoGCMTxQ,0.0,2004-10-19 21:33:08,0.0,1Iobyi_7BkFON25Oegs0aw,4.0,Love their subs. Cheap and top shelf ingredients.,0.0,nkN_do3fJ9xekchVC-v68A
3,oYMsq2Xvzw6UbrIlMWjb-A,0.0,2004-10-19 21:34:40,0.0,2F5J51OYtD49eyIUKJKVgg,4.0,Love their pizza. They used to have a great ta...,0.0,nkN_do3fJ9xekchVC-v68A
4,AtLv64FV-Pw6JuT3XUKU1g,0.0,2004-10-19 21:35:14,0.0,pho1XNCTeRxQVzWR_5vacg,4.0,Pokey Sticks are the best!,0.0,nkN_do3fJ9xekchVC-v68A
5,ydUqgWsF3F27TbauOyib0w,1.0,2004-12-19 20:47:24,1.0,Ef1skKLKZ9izwBmreb_-qw,4.0,"Frequently busy due to their great food, but t...",1.0,62GNFh5FySkA3MbrQmnqvg
6,N2PlDjUJVfOJzsPzY0Au1w,0.0,2004-12-19 20:56:54,0.0,6POnAs_4MijROSKeOevXHQ,3.0,Not the best part of town. Not particularly g...,0.0,62GNFh5FySkA3MbrQmnqvg
7,ikubvyZFO0kxhA56RETzIg,0.0,NaN,1.0,mNNTXbRPA6xKsWAFEJekdA,5.0,"Excellent compounding pharmacy, affectionately...",1.0,23J4vG9_xxxdnmi8CBX7Ng
8,ITieHQ8UwKq74FnqTKnPOQ,1.0,2005-03-14 18:50:04,2.0,agDLOa-3a8tA6o_S-4fGfQ,5.0,The coolest corner store in all of the world. ...,2.0,GXn4ZsasLKh0qZ5g3nIqcQ
9,uoUa8ugZLrly0bA268IqEg,1.0,2005-03-14 18:52:13,0.0,rnWCfxWkA10VP7QKj2z3wg,5.0,"Oh man, where else can you get a blueberry mil...",0.0,GXn4ZsasLKh0qZ5g3nIqcQ



### Exercise 1:

Given what we know about the data so far, describe a sensible interpolation method that we can use.

**Answer.**

If we assume that ...


Here is some code that implements our idea above:

In [6]:
nan_rows = reviews[reviews['date'].isnull()] # all rows where I am missing a date
date_format = '%Y-%m-%d %H:%M:%S'

for index, row in nan_rows.iterrows():
    previous_date = reviews.iloc[index - 1]['date']
    if isinstance(previous_date, str): # if the date of the previous row is not missing
        previous_date = datetime.datetime.strptime(previous_date, date_format) # let's format it correctly
    next_date = None
    next_date_count = 1
    while next_date is None: # keep trying to look at the next_date if it's still missing
        try:
            next_date = datetime.datetime.strptime(reviews.iloc[index + next_date_count]['date'], date_format)
        except Exception:
            next_date_count += 1
    difference = (next_date - previous_date).seconds
    current_date = previous_date + datetime.timedelta(seconds=difference/(next_date_count + 1))
    reviews.loc[index, 'date'] = current_date

## `star`

The next feature which has null values to handle is `star`. `star` is a numeric value and can range from 1 to 5, where 1 represents the lowest rating and 5 the highest. 

### Exercise 2:

Describe an appropriate method for filling in the missing values of `star`.

Impute the `star` rating based on columns that you do observe (cool, funny, useful, text). Sentiment analysis on the `text` feautre in partciular might be useful!

**Answer.**

What about interpolation using mean/median `star` value?
Pro: simple!
Con: introduce (perpetuate) bias, falsely deflate the variance, so be cautious about using this if you might be doing hypothesis testing/feaure importance analysis, all of these things require an accurate understanding of the...

Interpolation via model from known values: 

Impute the `star` rating based on columns that you do observe. (`cool`, `funny`, `useful`, `text`). 
Sentiment analysis on the `text` feature in particular might be useful!

start = NA <-- ?
cool = 0
funny = 0
useful = 1
text = "This wasn't that great of a restaurant because I had to wait in line for 20 min longer that what they told me!"

start = 2
cool = 0
funny = 0
useful = 1
text = "This was annoying because I had to wait so long!"

Y ~ f(x)
model #1 (sentiment analysis)
`sentiment_of_text` ~ `text`
categorical("positive", "negative", "neutral", "is_it_missing")

model #2 
`star` ~ f(`cool`, `funny`, `useful`, `sentiment_of_text`, `business_id`, `date`)

Maybe we want to assume `star` ratings stay quite stable for a given businees day-over-day HOWEVER, they could change a lot year-over-year, and so we could use `star` ratings for a given business on the same day/week/month to interpolate the `star` ratings for reviews where they are missing.

data cleaning --> data exploration --> data modeling --> data training --> deploy...

What can we understand about *how* a value is missing?

## `cool`, `funny`, `useful` and revisiting interpolation

We can apply a similar logic to conclude that the `cool`, `funny`, and `useful` columns should also be filled with `NaN`s:

In [7]:
reviews['cool'].fillna(np.nan, inplace=True)
reviews['funny'].fillna(np.nan, inplace=True)
reviews['useful'].fillna(np.nan, inplace=True)

Now, let's assume, for the sake of progress, that the inputs `cool`, `date`, `funny`, `useful`, and `text` ARE good predictors for `star`, and that the client is mostly interested in high-level properties of the distributions of these variables and how they correlate to each other. In such a case, we can go ahead and leverage the power of machine learning to predict a reasonable estimate for `star`. 

We will do so by using the [IterativeImputer](https://scikit-learn.org/stable/modules/impute.html#multivariate-feature-imputation) class of `scikit-learn` to fill the missing values in these columns. `IterativeImputer` fills one column after another by building models for each column:

In [8]:
reviews.isnull().any()

business_id    False
cool            True
date           False
funny           True
review_id      False
stars           True
text           False
useful          True
user_id        False
dtype: bool

In [9]:
imputer = IterativeImputer() # TODO: research what this does...

cols_to_impute = ['stars', 'cool', 'funny', 'useful']

imputed_df = pd.DataFrame(imputer.fit_transform(reviews[cols_to_impute])) # fit the transformation and then cast it as a DF
imputed_df
imputed_df.columns = cols_to_impute

reviews[['stars']] = imputed_df['stars'] # could be reviews[cols_to_impute] = imputed_df[cols_to_impute]
reviews.isnull().any()

business_id    False
cool            True
date           False
funny           True
review_id      False
stars          False
text           False
useful          True
user_id        False
dtype: bool

The value has to be dynamically filled, similar to the cool and funny columns.

### Dealing with erroneous values

Of course, missing values are not the only problem in datasets – erroneous values are too! So we should look at the other columns with non-null values to see if they could have errors that we can correct. Here, the other columns are `business_id`, `review_id`, `user_id`, and `text`. However, IDs are arbitrary identifiers which we have no way of ascertaining if they are correct or not, so we have to take the values we are given for granted.

`text` is the text of the user review, but the objective of any attempts to clean this is unclear - what metrics would we use to determine if a review text is "clean" or not? The answer to this question is not obvious at all without much more precise direction from the client about what they intend to use this text for, so we will leave it alone for now.

## Cleaning up `businesses`

Let's start by checking which columns contain null values:

In [10]:
businesses.isnull().any()

business_id    False
categories      True
city           False
latitude       False
longitude      False
name           False
dtype: bool

We see that there are null values in `categories`.

### Exercise 3:

Suppose for a moment that there were missing values in the `city` column. Can you describe a method which would allow us to effectively fill in missing `city` values?

**Answer.**

If for a given row, we ahave `latitude` and `longtitude`, then we can probably impute `city`!

`city` = NA
`latitude` = 34.19212
`longitude` = 20.29382
 
 What sort of Lookup table do you need?
 `city` `latitude` `longitude`
 
 And if there's a `city` missing from our Lookup table: (we could still impute by simply looking at the closest `longitude` and `latitude` in our `business` table)
 
 
How do we define closest?

m = city is missing
(lat_m, lon_m)

a = another row where city is not missing
(lat_a, lon_a)

Calculate the Euclidian distance!
d = sqrt{ (lat_m - lat_a)^2 + (lon_m - lon_a)^2 }

and then calculate d for all rows where city is not missing and pick the row where d is smallest! 
 

### Exercise 4:

Describe and implement the best method to fill in the null values in the `categories` column.

**Answer.**

Some ideas: 
1. We can ise if there's information in the business name that suggests the category (e.g. "Sunset Golf" is probably related to "Golf")
3. If the business name is missing we can join the business_id to the `reviews` table use the review's `text` to see if there's a mention of categories (e.g. "The food tasted great")

It should be noted that this is not an ideal solution as those businesses do have a category associated with them; we just do not know what it is. It could be random that some businesses were not documented with a category but it is also possible that we are dealing with a systematic error here. For example, perhaps all the businesses in a specific area were not documented. Our default solution is the best option we have, but a good data scientist should note when his/her imputation methods are subject to the possibility of high errors and what additional information they would need to address those.

### Another look at erroneous values

Of course, missing values are not the only thing that can go wrong in a dataset – erroneous values can affect the analysis as well. Looking at the columns which we have not dealt with yet, we have `business_id`, `latitude`, `longitude`, and `name` as potential candidates.

### Exercise 5:

Describe and implement suitable methods to deal with potential erroneous values in these columns.

**Answer.**

What if `business_id` is wrong? 
- The main issue I'd be worried about is the same business somehow getting entered twice i.e. McDonalds vs mcdonalds
- How could I check for duplicates?
- Use fuzzy match on `longitude`/ `latitude`/`name`  # TODO: reserach 'fuzzy' match!
- Ex: `Felix Trust Financial Advisors`, Boise, Idaho --> uncommon, probably a duplicate of in table more than once 

What if `latitude`, `longtitude` is wrong? 
- format is not lat/lon (e.g. impossible or location doesn't make sense, e.g. in the middle of the Pacific)
- if you have a `city`, you can also check if it matches the `city`
- if you know the business only operates in a certain country, then a location outside of that country doesn't make sense

What if `name` is wrong? 
- non-sensical characters ({, ], <, >, %)
- random character ('fdskkhsdf', 'sdfl')

## Adding information for `businesses`

On the client's request, we must find the county and state each business belongs in and add this to the table. We use the geoJSON files that contain the geoshapes of each state and county in the US. We will be using the [shapely](https://shapely.readthedocs.io/en/latest/manual.html) library to figure out whether a point is present inside the shape that represents the border of each state/county:

In [11]:
with open('us-state-shapes.json') as f:
    states = json.load(f)
    
def get_state_name(row):
    if not row['latitude'] or not row['longitude']:
        return None
    point = Point(row['longitude'], row['latitude'])
    for state in states['features']:
        polygon = shape(state['geometry'])
        if polygon.contains(point):
            return state['properties']['NAME']
        
businesses['state'] = businesses.apply(get_state_name, axis=1)

#generalization:
your_table['new_column_name'] = your_table.apply(your_function, axis=1)

NameError: name 'your_table' is not defined

### Exercise 6:

Find the county of each business and add them to the table.

**Answer.**

In [ ]:
with open('us-county-shapes.json') as f:
    counties = json.load(f)
    
def get_county_name(row):
    if not row['latitude'] or not row['longitude']:
        return None
    point = Point(row['longitude'], row['latitude'])
    for county in counties['features']:
        polygon = shape(county['geometry'])
        if polygon.contains(point):
            return county['properties']['NAME']
businesses['county'] = businesses.apply(get_county_name, axis=1)

How do I know the name of the geoJSON file I need to grab?
How do I know whether `county` objects have `geometry` and `properties` fields?
Check the `shapely` documentation!

In [ ]:
businesses.head(5)

We also need to add information about a restaurant's number of reviews and their average star rating.

### Exercise 7:

Calculate the number of reviews received for each restaurant. Remember, if you're using a truncated dataset, you may have restaurants with zero reviews.

**Answer.**

In [ ]:
def calculate_review_count(row):
    business_id = row["business_id"]
    business_reviews = reviews[reviews["business_id"] == business_id]
    return len(business_reviews)

businesses['review_count'] = businesses.apply(calculate_review_count, axis=1)


### Exercise 8:

For each restaurant, calculate its average star rating.

**Answer.**

In [ ]:
#HW 
def calculate_average_star(row):
    business_id = row["business_id"]
    business_reviews = reviews[reviews["business_id"] == business_id]
    return len(business_reviews)

## Aggregating the check-ins data

Now, let's do our final task: use the check-ins data given in the JSON file to compute various statistics on the number of check-ins over various time periods.

### Exercise 9:

Compute, for each business, the total number of check-ins by hour of day, by weekday, by day, and across all time.

**Answer.**

In [1]:
#HW

## Conclusions

In this case, we practiced our skills in dealing with missing values and also learned about common ways of handling erroneous values. We then creatively leveraged external data in order to engineer additional features based on client requests.

## Takeaways

In this case, you looked more at how to deal with missing data. You learned that although interpolation is a powerful tool, when and how to use it depends highly on the projected use cases of the data. Sometimes, it is better to simply replace a missing value with "Not found" or `NaN` so that it is clear to end users to discount it appropriately from subsequent analyses, rather than run the risk of skewing the resultant data distribution or summary statistics. Removing rows is generally okay in 2 cases: when your data is corrupted, or if the data has a marginal impact on what you are trying to do. For example, if you don't need a particular piece of data to do anything, it is easier to leave them as `NaN` instead of inputting them.

You also learned a few ways of using existing public data to help interpolate missing values in your current dataset. This is an invaluable skill and often times the best way to deal with missing values is to exercise this resourcefulness. It is important to recognize not all interpolation is made equal. If you have many parameters that are imputed poorly or with great uncertainty, it will be much harder to develop an accurate prediction model later if those parameters are key predictors.

Finally, you looked at erroneous values and various common ways these could leak into the data. You also learned that sometimes it is too difficult or impossible to systematically determine the existence of error.
    
We dove into the technical nuances of data cleaning in this case but it is important to note this dataset is very simple compared to the data you will be working with in real life. It is not too uncommon to deal with datasets with hundreds of parameters where dozens have different types of missing/incorrect data. Additionally, you may encounter datasets where you are unsure what some columns/parameters mean. This dataset came with a nice handy dictionary explaining what information is in each column. 
    
Students are highly encouraged to thoroughly review all the EDA-related cases we have taught up to this point. Mastery of EDA will immensely improve the quality of your data wrangling & cleaning process, which in combination will impact your subsequent modeling process in a positive way.